In [1]:
import numpy as np
from keras.datasets import imdb
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import LSTM, Dense, Embedding, Input, Dot, Flatten
from keras_preprocessing.sequence import pad_sequences
from tensorflow.python.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
import pandas as pd
import re

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
df = pd.read_csv('train.csv')

In [84]:
sentences = []
for i in range(df.shape[0]):
    temp = df['text'][i].lower()
    temp = re.sub(r'[^a-zA-Z\s]', ' ', temp)
    temp = temp.split()
    sentences.append(temp)

In [5]:
words = set(word for sentence in sentences for word in sentence)
word2idx = {word: i + 1 for i, word in enumerate(words)}
idx2word = {i: word for word, i in word2idx.items()}

In [6]:
def generate_training_data(sentences, word2idx, window_size, num_negative_samples):
    X, Y, vec = [], [], []
    index = 0
    for sentence in sentences:
        index += 1
        print(index)
        sentence = [word2idx[word] for word in sentence]
        for i, target_word in enumerate(sentence):
            for j in range(max(0, i - window_size), min(len(sentence), i + window_size + 1)):
                if i != j:
                    X.append(target_word)
                    Y.append(sentence[j])
                    vec.append(1)
                    negative_samples = np.random.choice(list(word2idx.values()), size=num_negative_samples)
                    for neg_word in negative_samples:
                        X.append(target_word)
                        Y.append(neg_word)
                        vec.append(0)
    return np.array(X), np.array(Y), vec

In [7]:
X, Y, vec = generate_training_data(sentences, word2idx, window_size=2, num_negative_samples=1)
V = len(words)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [8]:
X_train, X_val, Y_train, Y_val, vec_train, vec_val = train_test_split(X, Y, vec, test_size=0.3, random_state=42)

input_target = Input((1,))
input_context = Input((1,))

embedding = Embedding(V, 50, input_length=1, name='embedding')
target_embedding = embedding(input_target)
context_embedding = embedding(input_context)

dot_product = Dot(axes=-1)([target_embedding, context_embedding])
output = Flatten()(dot_product)
output = Dense(1, activation='sigmoid')(output)


model = Model(inputs=[input_target, input_context], outputs=output)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

checkpoint = ModelCheckpoint(filepath='RNN_test.h5',
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True
                             )

In [9]:
model.fit([X_train, Y_train], np.array(vec_train),
          validation_data=([X_val, Y_val], np.array(vec_val)),
         epochs=50, batch_size=4096, callbacks=[checkpoint])

Epoch 1/50
166/166 [==============================] - 1s 4ms/step - loss: 0.6456 - accuracy: 0.6950 - val_loss: 0.5480 - val_accuracy: 0.7838

Epoch 00001: val_accuracy improved from -inf to 0.78379, saving model to RNN_test.h5
Epoch 2/50
166/166 [==============================] - 1s 3ms/step - loss: 0.4824 - accuracy: 0.7994 - val_loss: 0.4643 - val_accuracy: 0.7949

Epoch 00002: val_accuracy improved from 0.78379 to 0.79490, saving model to RNN_test.h5
Epoch 3/50
166/166 [==============================] - 1s 3ms/step - loss: 0.4270 - accuracy: 0.8144 - val_loss: 0.4439 - val_accuracy: 0.8010

Epoch 00003: val_accuracy improved from 0.79490 to 0.80104, saving model to RNN_test.h5
Epoch 4/50
166/166 [==============================] - 1s 3ms/step - loss: 0.3982 - accuracy: 0.8257 - val_loss: 0.4308 - val_accuracy: 0.8060

Epoch 00004: val_accuracy improved from 0.80104 to 0.80598, saving model to RNN_test.h5
Epoch 5/50
166/166 [==============================] - 1s 3ms/step - loss: 0.372

In [10]:
weights = embedding.get_weights()[0]

In [85]:
sentences

[['our',
  'deeds',
  'are',
  'the',
  'reason',
  'of',
  'this',
  'earthquake',
  'may',
  'allah',
  'forgive',
  'us',
  'all'],
 ['forest', 'fire', 'near', 'la', 'ronge', 'sask', 'canada'],
 ['all',
  'residents',
  'asked',
  'to',
  'shelter',
  'in',
  'place',
  'are',
  'being',
  'notified',
  'by',
  'officers',
  'no',
  'other',
  'evacuation',
  'or',
  'shelter',
  'in',
  'place',
  'orders',
  'are',
  'expected'],
 ['people',
  'receive',
  'wildfires',
  'evacuation',
  'orders',
  'in',
  'california'],
 ['just',
  'got',
  'sent',
  'this',
  'photo',
  'from',
  'ruby',
  'alaska',
  'as',
  'smoke',
  'from',
  'wildfires',
  'pours',
  'into',
  'a',
  'school'],
 ['rockyfire',
  'update',
  'california',
  'hwy',
  'closed',
  'in',
  'both',
  'directions',
  'due',
  'to',
  'lake',
  'county',
  'fire',
  'cafire',
  'wildfires'],
 ['flood',
  'disaster',
  'heavy',
  'rain',
  'causes',
  'flash',
  'flooding',
  'of',
  'streets',
  'in',
  'manitou',
 

In [75]:
maxWords = 0
for i in range(len(sentences)):
    if maxWords < len(sentences[i]):
        maxWords = len(sentences[i])

In [90]:
train_sentences = np.zeros((df.shape[0], maxWords, 50))
                           
for i in range(len(sentences)):
    for j in range(len(sentences[i])):
        train_sentences[i][j] = weights[word2idx[sentences[i][j]] - 1]

In [98]:
# test = pd.read_csv('test.csv')
# test_sentences = []
# for i in range(df.shape[0]):
#     temp = df['text'][i].lower()
#     temp = re.sub(r'[^a-zA-Z\s]', ' ', temp)
#     temp = temp.split()
#     test_sentences.append(temp)

In [99]:
# for i in range(len(test_sentences) - 1):
#     for j in range(len(test_sentences[i]) - 1):
#         test_sentences[i][j] = weights[word2idx[test_sentences[i][j]]]

In [63]:
targets = []
for i in range(df['target'].shape[0]):
    targets.append(df['target'][i])
targets = np.array(targets)

In [26]:
model = Sequential()
model.add(LSTM(50, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [102]:
x_train, x_rest, y_train, y_rest = train_test_split(train_sentences, targets, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(train_sentences, targets, test_size=0.5, random_state=42)

In [103]:
batch_size = 2048
epochs = 100
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=epochs, batch_size=batch_size)

Epoch 1/100
3/3 [==============================] - 1s 197ms/step - loss: 0.5107 - accuracy: 0.7546 - val_loss: 0.4533 - val_accuracy: 0.7916
Epoch 2/100
3/3 [==============================] - 1s 174ms/step - loss: 0.5126 - accuracy: 0.7497 - val_loss: 0.4563 - val_accuracy: 0.7869
Epoch 3/100
3/3 [==============================] - 0s 167ms/step - loss: 0.5110 - accuracy: 0.7519 - val_loss: 0.4645 - val_accuracy: 0.7840
Epoch 4/100
3/3 [==============================] - 0s 172ms/step - loss: 0.5125 - accuracy: 0.7472 - val_loss: 0.4581 - val_accuracy: 0.7953
Epoch 5/100
3/3 [==============================] - 0s 173ms/step - loss: 0.5042 - accuracy: 0.7568 - val_loss: 0.4513 - val_accuracy: 0.7951
Epoch 6/100
3/3 [==============================] - 0s 169ms/step - loss: 0.5068 - accuracy: 0.7523 - val_loss: 0.4465 - val_accuracy: 0.7985
Epoch 7/100
3/3 [==============================] - 0s 173ms/step - loss: 0.5091 - accuracy: 0.7527 - val_loss: 0.4503 - val_accuracy: 0.7964
Epoch 8/100
3

In [95]:
model.evaluate(x_test, y_test, verbose=

array([[[-0.18728517,  0.67523348,  0.18378635, ..., -0.1585082 ,
          0.25586227,  0.01768778],
        [ 0.10583924, -0.38248336,  0.11513151, ..., -0.05152744,
          0.46629408,  0.2864925 ],
        [ 0.08076834,  0.4025124 , -0.50340503, ...,  0.12507363,
          0.17569141, -0.15416989],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.07234139, -0.0409301 ,  0.25591952, ...,  0.0173683 ,
         -0.3228977 ,  0.31163514],
        [-0.37176731, -0.00796524,  0.31618395, ..., -0.23002866,
         -0.15891586, -0.24058598],
        [ 0.00258643,  0.29322168,  0.01925898, ...,  0.07202357,
         -0.05753399,  0.07236768],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  